In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [2]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_uppercase,pide_hombre,pide_mujer,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR,True,False,False,...,True,False,False,True,False,False,False,False,False,False
1,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,True,False,False,...,False,True,False,True,False,False,False,False,False,False
2,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,True,False,False,...,False,False,False,True,False,False,False,False,False,False
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,True,False,False,...,False,False,False,True,False,False,False,False,False,False


In [3]:
len(avisos)

25288

In [4]:
#vamos a intentar algunos features haciendo un analisis lexico del titulo del aviso.
#vemos cuales son las palabras más utilizadas en el id_aviso.
avisos.titulo.str.split(expand=True).stack().value_counts()

de                 13293
-                   6364
Analista            2959
para                2787
y                   1896
en                  1780
DE                  1654
Zona                1331
/                   1010
Técnico              945
con                  912
Administrativo       847
Vendedor             783
Comercial            742
Asistente            713
Jefe                 651
Ventas               643
Contable             595
CABA                 564
Supervisor           562
Ejecutivo            529
a                    503
Responsable          465
Operario             453
zona                 447
Ingeniero            423
al                   423
Senior               397
Importante           383
PARA                 380
                   ...  
ani                    1
Seller                 1
Mysql                  1
Automatización.        1
CD                     1
CONTACTO               1
pino                   1
(Store                 1
(Corrientes)           1


In [5]:
import re
def contiene_frase(texto, frase):
    if re.search(frase, texto,re.IGNORECASE):
        return True
    return False
contiene_frase=np.vectorize(contiene_frase)

In [6]:
#agrego campo vendedor.
avisos['pide_vendedor'] = (contiene_frase(avisos['titulo'], "vendedor") | contiene_frase(avisos['titulo'], "vendedora")
                          | contiene_frase(avisos['titulo'], "vendedor/a"))

In [7]:
#agrego campo administrativo.
avisos['pide_administrativo'] = (contiene_frase(avisos['titulo'], "administrativo") | contiene_frase(avisos['titulo'], "administrativa"))

In [8]:
#agrego campo tecnico
avisos['pide_tecnico'] = (contiene_frase(avisos['titulo'], "Técnico"))

In [9]:
#agrego campo analista
avisos['pide_analista'] = (contiene_frase(avisos['titulo'], "analista"))

In [10]:
#agrego campo comercial
avisos['pide_comercial'] = (contiene_frase(avisos['titulo'], "comercial"))

In [11]:
#agrego campo asistente
avisos['pide_asistente'] = (contiene_frase(avisos['titulo'], "asistente"))

In [12]:
#agrego campo ejecutivoo
avisos['pide_ejecutivo'] = (contiene_frase(avisos['titulo'], "ejecutivo"))

In [13]:
#agrego campo ingeniero
avisos['pide_ingeniero'] =(contiene_frase(avisos['titulo'], "ingeniero"))

In [14]:
#agrego campo operario
avisos['pide_operario'] = (contiene_frase(avisos['titulo'], "operario"))

In [15]:
#agrego campo desarrolador
avisos['pide_desarollador'] = (contiene_frase(avisos['titulo'], "desarrollador"))

In [16]:
print(sum(avisos['pide_vendedor']))
print(sum(avisos['pide_administrativo']))
print(sum(avisos['pide_tecnico']))
print(sum(avisos['pide_analista']))
print(sum(avisos['pide_comercial']))
print(sum(avisos['pide_asistente']))
print(sum(avisos['pide_ejecutivo']))
print(sum(avisos['pide_ingeniero']))
print(sum(avisos['pide_operario']))
print(sum(avisos['pide_desarollador']))

1870
1823
1313
3339
1317
831
726
620
925
549


In [16]:
avisos.to_csv("../../Data/fiuba_entrenamiento/gian/avisos_titulos.csv")

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_uppercase,pide_hombre,pide_mujer,...,pide_vendedor,pide_administrativo,pide_tecnico,pide_analista,pide_comercial,pide_asistente,pide_ejecutivo,pide_ingeniero,pide_operario,pide_desarollador
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR,True,False,False,...,True,False,False,False,False,False,False,False,False,False
1,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,True,False,False,...,True,False,False,False,False,False,False,False,False,False
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,True,False,False,...,True,False,False,False,False,False,False,False,False,False


In [18]:
len(avisos)

25288